## DIA 111: Agregar un Código QR de Descarga y Botones en Telegram para Ver el Reporte


Hoy vas a dar un toque profesional a tu sistema de reportes: cada vez que se genere uno, el usuario podrá:

📲 Recibir un mensaje de Telegram con botones interactivos

📷 Recibir un QR escaneable que lleva al reporte PDF/CSV

📁 Descargar su reporte desde un enlace seguro

✅ ¿Qué aprenderás hoy?

📦 Subir el reporte a un servidor y generar un enlace.

📸 Crear un código QR con qrcode.

🤖 Enviar botones en Telegram usando inline_keyboard.

🧩 1. Instalar librería qrcode
bash
Copiar
Editar
pip install qrcode[pil]
🧩 2. Generar y guardar código QR desde una URL
python
Copiar
Editar
import qrcode

def generar_qr_a_pdf(url_descarga, nombre_archivo='qr_temp.png'):
    qr = qrcode.QRCode(box_size=10, border=2)
    qr.add_data(url_descarga)
    qr.make(fit=True)

    img = qr.make_image(fill='black', back_color='white')
    img.save(nombre_archivo)
    return nombre_archivo
🧩 3. Subir el PDF/CSV generado a tu servidor
En Render o cualquier hosting que uses, puedes guardar el archivo en un folder público o usar servicios como Firebase, Amazon S3 o directamente una ruta Flask.

Por ejemplo:

python
Copiar
Editar
ruta_archivo = f'static/reportes/reporte_{user.username}.{ext}'
with open(ruta_archivo, 'wb') as f:
    f.write(reporte.read())

url_publica = f"https://tu-dominio.com/{ruta_archivo}"
🧩 4. Enviar mensaje de Telegram con botón y QR
python
Copiar
Editar
def enviar_mensaje_con_qr(chat_id, url_reporte, username):
    TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
    qr_path = generar_qr_a_pdf(url_reporte)

    mensaje = f"""
📊 *Hola {username}*  
Tu reporte está listo. Puedes descargarlo desde el botón o escaneando el código QR. ✅
"""

    # 1. Enviar mensaje con botón
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": mensaje,
        "parse_mode": "Markdown",
        "reply_markup": {
            "inline_keyboard": [[{
                "text": "📥 Descargar Reporte",
                "url": url_reporte
            }]]
        }
    }

    res1 = requests.post(url, json=payload)

    # 2. Enviar imagen QR
    url_img = f"https://api.telegram.org/bot{TOKEN}/sendPhoto"
    with open(qr_path, 'rb') as img_file:
        res2 = requests.post(url_img, files={'photo': img_file}, data={'chat_id': chat_id})

    if res1.status_code == 200 and res2.status_code == 200:
        print("✅ Mensaje con QR y botón enviado")
    else:
        print("❌ Error al enviar:", res1.text, res2.text)
